## Data Collection and Preprocessing (2+3+1 = 6 marks)
- Collect relevant datasets.
- Preprocess the text data that will involve different NLP tasks where needed.
- Split the dataset into a training set and a test set.


## Feature Extraction and Feature Engineering (3+3 = 6 marks)
- Extract and select numerical features from text data using NLP techniques - Bag of Words (BoW), TF-IDF, or word embeddings (Word2Vec, GloVe).

## Model Selection (6 marks)
- Feedback/Sentiment Analysis: Choose a classification model like Naive Bayes, Logistic Regression, Random Forest, or neural networks.
- Topic Modeling: Use unsupervised models like clustering,  LDA, or other unsupervised methods.


## Model Training and Evaluation (2 + 2 + 2 = 6 marks)
- Train the model on the training dataset.
- Evaluate the model using appropriate metrics:
  - Classification: Accuracy, Precision, Recall, F1-Score.
  - Topic Modeling: Perplexity, Topic Coherence.
- Fine-tune model parameters for optimal performance.


## Model Refinement and Optimization (3+3 = 6 marks)
- Experiment with different autoencoder architectures, hyperparameters, and reconstruction error thresholds to fine-tune the model's performance.
- Apply techniques such as learning rate scheduling, regularization, and dropout to improve the model's generalization and robustness.

## Documentation (10 marks) and 15 mins Presentation (10 marks)
Document the entire project, including the dataset description, preprocessing steps, model architecture, hyperparameter settings, and evaluation results.

Create a presentation summarizing the project's objectives, methods, findings, and recommendations.